# Classificação de documentos - bag-of-words

Este documento implementa um pipeline para classificação de documentos usando [técnicas "tradicionais" de machine learning com a biblioteca scikit-learn](http://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html).

### Referências
* [Classification of text documents using sparse features](http://scikit-learn.org/stable/auto_examples/text/document_classification_20newsgroups.html)
* [Text Classification with NLTK and Scikit-Learn](http://bbengfort.github.io/tutorials/2016/05/19/text-classification-nltk-sckit-learn.html)
* [Working With Text Data](http://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html)
* [Analyzing tf-idf results in scikit-learn](https://buhrmann.github.io/tfidf-analysis.html)

### Importando bibliotecas

In [157]:
%reload_ext autoreload
%autoreload 1

%aimport dataset

import numpy as np
import scipy.sparse as sp
import pandas as pd
import random

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_predict, cross_val_score, GridSearchCV

### Carregando dados de normas

In [2]:
print("Carregando normas...")
normas = pd.read_csv('data/normas.csv')
print("%d documentos" % normas.shape[0])
print("%d categorias" % len(normas.IDETEMA.unique()))

Carregando normas...
257178 documentos
43 categorias


### Selecionando normas com tema associado

In [3]:
normas_com_tema = normas[normas.IDETEMA.notnull()].copy()
print("%d normas com tema" % normas_com_tema.shape[0])
print("%d categorias" % len(normas_com_tema.CLASS.unique()))

32681 normas com tema
42 categorias


### Tratamento de temas obsoletos

In [4]:
def merge_labels(data, new_label, old_label):
    old_class = data[data.TEMA == old_label].CLASS
    
    if len(old_class) > 0:
        old_class = old_class.values[0]
        new_class = data[data.TEMA == new_label].CLASS.values[0]
        data['TEMA'] = data.TEMA.map(lambda x: new_label if x == old_label else x)
        data['CLASS'] = data.CLASS.map(lambda x: new_class if x == old_class else x)
        sorted_classes = np.sort(data.CLASS.unique())
        data['CLASS'] = data['CLASS'].map(lambda x: np.where(sorted_classes == x)[0][0])
    
merge_labels(normas_com_tema, 'Ciência e Tecnologia', 'Ciência, Tecnologia e Informática')
merge_labels(normas_com_tema, 'Indústria, Comércio e Abastecimento', 'Indústria, Comércio e Defesa do Consumidor')
merge_labels(normas_com_tema, 'Organização Administrativa do Estado', 'Organização Político-Administrativa do Estado')
print("%d normas com tema" % normas_com_tema.shape[0])
print("%d categorias" % len(normas_com_tema.CLASS.unique()))
normas_com_tema.head()

32681 normas com tema
39 categorias


,IDENORMA,IDETIPONORMA,IDESITUACAONORMA,TEXNUMERONORMA,ANONORMA,NUMSESSAOLEGISLATIVA,DATASSINATURA,TEXAPELIDO,TEXCOLECAO,TEXORIGENS,TEXSIGLASORIGENS,TEXEPIGRAFE,TEXEPIGRAFEREFERENCIA,NOMTIPONORMA,SIGTIPONORMA,TEXTO,INDEXACAO,IDETEMA,TEMA,CLASS
12,609841,33828,NaN,54560,2010,NaN,2010-12-27 00:00:00,NaN,Legislação Superior,Poder Executivo,PE,Decreto de 27 de Dezembro de 2010,Decreto de 27 de Dezembro de 2010,Decreto Sem Número,decret_sn,"O PRESIDENTE DA REPUBLICA, no uso ...",REFORMA AGRARIA - Imovel rural - Desapropriaca...,51.0,Política Fundiária,27
45,605183,33823,NaN,7149,2010,NaN,2010-04-08 00:00:00,NaN,Legislação Superior,Poder Executivo,PE,"Decreto nº 7.149, de 8 de Abril de 2010",Decreto nº 7149 de 8 de Abril de 2010,Decreto,decret,"O PRESIDENTE DA REPUBLICA,...",ATOS INTERNACIONAIS - Republica Democratica do...,55.0,Relações Internacionais,31
54,599446,33823,NaN,7059,2009,NaN,2009-12-29 00:00:00,NaN,Legislação Superior,Poder Executivo,PE,"Decreto nº 7.059, de 29 de Dezembro de 2009",Decreto nº 7059 de 29 de Dezembro de 2009,Decreto,decret,O PRESIDENTE DA RE...,SEGURO RURAL - Subvencao economica - Premio - ...,51.0,Política Fundiária,27
74,609851,33828,NaN,54565,2010,NaN,2010-12-27 00:00:00,NaN,Legislação Superior,Poder Executivo,PE,Decreto de 27 de Dezembro de 2010,Decreto de 27 de Dezembro de 2010,Decreto Sem Número,decret_sn,"O PRESIDENTE DA REPUBLICA,...",REFORMA AGRARIA - Imovel rural - Desapropriaca...,51.0,Política Fundiária,27
85,775237,33823,NaN,7906,2013,NaN,2013-02-04 00:00:00,NaN,Legislação Superior,Poder Executivo,PE,"Decreto nº 7.906, de 4 de Fevereiro de 2013",Decreto nº 7906 de 4 de Fevereiro de 2013,Decreto,decret,"A PRESIDENTA DA REPUBLICA, no uso da atri...",ATOS INTERNACIONAIS - Reino dos Paises Baixos ...,43.0,Direito Penal e Processual Penal,15


### Salvando tabela de temas

In [175]:
temas = normas_com_tema[['TEMA', 'CLASS']].drop_duplicates().sort_values('CLASS')
temas.to_csv('models/temas.csv', index=False, encoding='utf-8')

### Train/test split

In [170]:
X_train, X_test, y_train, y_test = train_test_split(normas_com_tema.TEXTO, normas_com_tema.CLASS, 
                                                    test_size=0.1, random_state=42, stratify=normas_com_tema.CLASS)

print('Train samples:', X_train.shape[0])
print('Test samples:', X_test.shape[0])
print('Labels:', len(y_train.unique()))

Train samples: 29412
Test samples: 3269
Labels: 39


### Treinamento e avaliação do modelo

In [171]:
def train(X_train, y_train, grid_search=True, cv=3):

    pipeline = Pipeline([
        #('vect', CountVectorizer(ngram_range=(1, 2), max_df=0.75)),
        #('tfidf', TfidfTransformer()),
        ('tfidf', TfidfVectorizer(ngram_range=(1, 2), max_df=0.75)),
        ('clf', SGDClassifier(class_weight='balanced', 
                              loss='log', n_jobs=2, n_iter=50,
                              alpha=1e-5)),
    ])

    if grid_search:
    
        # uncommenting more parameters will give better exploring power but will
        # increase processing time in a combinatorial way
        parameters = {
            'vect__max_df': (0.5, 0.75, 1.0),
            #'vect__max_features': (None, 5000, 10000, 50000),
            #'vect__ngram_range': ((1, 1), (1, 2)),  # unigrams or bigrams
            #'tfidf__use_idf': (True, False),
            #'tfidf__norm': ('l1', 'l2'),
            'clf__alpha': (0.00001, 0.000001),
            'clf__penalty': ('l2', 'elasticnet'),
            #'clf__n_iter': (10, 50, 80),
        }

        # find the best parameters for both the feature extraction and the
        # classifier
        grid_search = GridSearchCV(pipeline, parameters, n_jobs=-1, verbose=1, cv=cv)

        print("Performing grid search...")
        print("pipeline:", [name for name, _ in pipeline.steps])
        print("parameters:")
        print(parameters)

        grid_search.fit(X_train, y_train)

        print("Best score: %0.3f" % grid_search.best_score_)
        print("Best parameters set:")
        best_parameters = grid_search.best_estimator_.get_params()
        for param_name in sorted(parameters.keys()):
            print("\t%s: %r" % (param_name, best_parameters[param_name]))

        return grid_search
    
    else:
        pipeline.fit(X_train, y_train)
        scores = cross_val_score(pipeline, X_train, y_train, cv=cv)
        print("Scores: ", scores)
        return pipeline


def evaluate(clf, X_test, y_test, top_k=3, confusion=False):
        
    print("Predicting the outcomes of the testing set")

    pred = clf.predict(X_test)
    
    print("Classification report on test set for classifier:")
    #print(clf)
    print()
    print(classification_report(y_test, pred))
    
    if confusion:
        print()
        print("Confusion matrix:")
        print(confusion_matrix(y_test, pred))
    
    probs = clf.predict_proba(X_test)
    best_k = np.argsort(probs, axis=1)[:,-top_k:]
    pred_k = [y_true if y_true in pred else pred[-1] for y_true, pred in zip(y_test, best_k)]

    print()
    print("Classification report on test set - TOP", top_k)
    #print(clf)
    print()
    print(classification_report(y_test, pred_k))
    
    if confusion:
        print()
        print("Confusion matrix - TOP", top_k)
        print(confusion_matrix(y_test, pred_k))

In [172]:
%%time
clf = train(X_train, y_train, grid_search=False)

Scores:  [ 0.70044816  0.70692645  0.69839649]
Wall time: 16min 15s


In [173]:
%%time
evaluate(clf, X_test, y_test, top_k=4)

Predicting the outcomes of the testing set
Classification report on test set for classifier:

             precision    recall  f1-score   support

          0       0.72      0.39      0.51       468
          1       0.32      0.67      0.43        27
          2       0.43      0.53      0.48        43
          3       0.19      0.43      0.27        14
          4       0.20      0.40      0.27         5
          5       1.00      0.98      0.99       707
          6       0.24      0.43      0.31        14
          7       0.40      0.48      0.44        29
          8       0.25      0.41      0.31        17
          9       0.38      0.67      0.48        12
         10       0.56      0.62      0.59        24
         11       0.50      0.75      0.60         4
         12       0.22      1.00      0.36         2
         13       0.33      0.40      0.36         5
         14       0.00      0.00      0.00         1
         15       0.40      0.42      0.41        24
    

C:\Users\p_7174\AppData\Local\Continuum\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\metrics\classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### Salvando o modelo treinado

In [174]:
from sklearn.externals import joblib
import os

def save_model(model, path):
    
    if not os.path.exists(os.path.dirname(path)):
        try:
            os.makedirs(os.path.dirname(path))
        except OSError as exc: # Guard against race condition
            if exc.errno != errno.EEXIST:
                raise

    joblib.dump(model, path)
    
save_model(clf, 'models/bag-of-words-all-classes.pkl')